In [5]:
import tensorflow_model_optimization as tfmot;
import tensorflow as tf;
import numpy as np
import time

2022-05-25 10:28:54.537088: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-25 10:28:54.537130: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
print("================ Data Loading ================")
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train = np.reshape(x_train, (-1,28, 28, 1))
x_test = np.reshape(x_test, (-1, 28, 28, 1))

# Data shapes
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)
print("")

================ Data Loading ================
x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)
y_train shape: (60000,)
y_test shape: (10000,)



## Lenet5

2022-05-25 10:29:01.597554: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-25 10:29:01.597608: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-25 10:29:01.597637: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (big25): /proc/driver/nvidia/version does not exist
2022-05-25 10:29:01.599285: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()],
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

model.fit(
    x_train,
    y_train,
    epochs=10,
    batch_size=64,
    validation_data= (x_test, y_test),
 )

Epoch 1/10


/home/karimhocine/.local/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


938/938 [==============================] - 10s 10ms/step - loss: 0.2994 - sparse_categorical_accuracy: 0.9099 - val_loss: 0.0930 - val_sparse_categorical_accuracy: 0.9709
Epoch 2/10
938/938 [==============================] - 10s 10ms/step - loss: 0.0820 - sparse_categorical_accuracy: 0.9750 - val_loss: 0.0697 - val_sparse_categorical_accuracy: 0.9789
Epoch 3/10
938/938 [==============================] - 9s 10ms/step - loss: 0.0586 - sparse_categorical_accuracy: 0.9816 - val_loss: 0.0519 - val_sparse_categorical_accuracy: 0.9830
Epoch 4/10
938/938 [==============================] - 9s 10ms/step - loss: 0.0446 - sparse_categorical_accuracy: 0.9860 - val_loss: 0.0407 - val_sparse_categorical_accuracy: 0.9868
Epoch 5/10
938/938 [==============================] - 9s 10ms/step - loss: 0.0381 - sparse_categorical_accuracy: 0.9880 - val_loss: 0.0393 - val_sparse_categorical_accuracy: 0.9872
Epoch 6/10
938/938 [==============================] - 9s 10ms/step - loss: 0.0324 - sparse_categorical_a

In [10]:
model_quantifie = tf.keras.models.clone_model(model)

## Post Training Quantization

In [124]:
def  matrix2vect(w):
    # Vectorisation de la matrice des poids
    layer_shapes = []
    vect = []

    for i in range (len(w)//2):
      # rassembler les poids et biais dans une seule matrices
      v = tf.concat(  [tf.reshape(w[2*i],( -1,1 )), tf.reshape(w[2*i+1], (-1,1)) ], axis=0)

      # enregistrer les dimensiosn des matrices pour effectuer l'opération inverse
      layer_shapes.append(v.shape)

      # Vectoriser les matrices de poids-biais
      vect.append(tf.reshape(v, (-1, 1)))
    
    return vect, layer_shapes
                                               
def vect2matrix(self, v):
    w = []
    # Cette fonction permet de transformer les poids quantifiées sous le formats correspondants aux dimensions des poids des couches
    for i in range(len(v)):
        mat = tf.reshape(v[i], self.layer_shapes[i])
        w.append(mat[0:-1])
        w.append(mat[-1])
    return w


  # Transforme les valeurs d'un vecteur vers l'intervalle [0, 1]
def scale_function(tab, bucket_size):

  Vecteur = []
  A = []
  B = []

  for tab in tab:
        tab = tf.reshape(tab, (-1))
        if bucket_size > tab.shape[0]:
          raise ValueError(f'Bucket_size ({bucket_size}) must be smaller than or equal to the vector length ({len(tab)})')
        v = tf.constant([])
        alpha = []
        beta = []

        nb_bucket = tab.shape[0]//bucket_size

        # Nombre de bucket pair
        if tab.shape[0] % bucket_size == 0:
          nb_param = nb_bucket


          for i in range(nb_param):
            b = tf.math.reduce_min(tf.slice(tab[:], begin=tf.constant([i* bucket_size]), size= tf.constant([bucket_size]) ) )
            a = tf.math.reduce_max(tf.slice(tab[:], begin=tf.constant([i* bucket_size]), size= tf.constant([bucket_size]) ) ) - tf.math.reduce_min(tf.slice(tab[:], begin=tf.constant([i* bucket_size]), size= tf.constant([bucket_size]) ) )
            alpha.append(a)
            beta.append(b)

            """if a == 0:
                  a = tf.constant([1])"""
            vect = (tf.slice(tab[:], begin=tf.constant([i* bucket_size]), size= tf.constant([bucket_size]) ) - b) / a
            v = tf.concat([v, vect], 0)

        # Nombre de bucket impair
        else:
          nb_param = nb_bucket + 1

          for i in range(nb_param):
            if i == nb_param - 1:
              b = tf.math.reduce_min(tf.slice(tab[:], begin=tf.constant([i* bucket_size]), size= tf.constant([tab.shape[0] - i* bucket_size]) ) )
              a = tf.math.reduce_max(tf.slice(tab[:], begin=tf.constant([i* bucket_size]), size= tf.constant([tab.shape[0] - i* bucket_size]) ) ) - tf.math.reduce_min(tf.slice(tab[:], begin=tf.constant([i* bucket_size]), size= tf.constant([tab.shape[0] - i* bucket_size]) ) )
              alpha.append(a)
              beta.append(b)

              """if a == 0:
                  a = tf.constant([1])"""

              vect = (  tf.slice(tab[:], begin=tf.constant([i* bucket_size]), size= tf.constant([tab.shape[0] - i* bucket_size]))  - b) /(a)

            else:
              b = tf.math.reduce_min(tf.slice(tab[:], begin=tf.constant([i* bucket_size]), size= tf.constant([bucket_size]) ) )
              a = tf.math.reduce_max(tf.slice(tab[:], begin=tf.constant([i* bucket_size]), size= tf.constant([bucket_size]) ) ) - tf.math.reduce_min(tf.slice(tab[:], begin=tf.constant([i* bucket_size]), size= tf.constant([bucket_size]) ) )
              alpha.append(a)
              beta.append(b)

              """if a == 0:
                  a = tf.constant([1])"""

              vect = (tf.slice(tab[:], begin=tf.constant([i* bucket_size]), size= tf.constant([bucket_size]) ) - b) / a
            v = tf.concat([v, vect], 0)

        A.append(alpha)
        B.append(beta)
        Vecteur.append(v) 

  return Vecteur, A, B

def uniform_quantification( Vect, nb_bits):
    v_q = []
    s = tf.constant([2**nb_bits], dtype=tf.float32)

    for v in Vect:
        k = tf.math.subtract(tf.math.multiply(v,s) , tf.math.floor( tf.math.multiply(v,s)) )
        eps = tf.ones(k.shape[0])

        eps = tf.where( tf.greater(k,0.5),eps, 0 )
        v_s = tf.math.multiply(v,s)
        res =  tf.math.floor(tf.math.divide(v_s, s))

        Q =  res + (tf.math.divide(eps ,s) )

        v_q.append(Q)
    return v_q

### Transformer les poids en vecteur

In [125]:
w = model.weights
original_shapes = []
for W in w:
    print(W.shape)
    original_shapes.append(W.shape)
    
    
vect, layer_shapes = matrix2vect(w)
layer_shapes

(5, 5, 1, 6)
(6,)
(5, 5, 6, 16)
(16,)
(256, 120)
(120,)
(120, 84)
(84,)
(84, 10)
(10,)


[TensorShape([156, 1]),
 TensorShape([2416, 1]),
 TensorShape([30840, 1]),
 TensorShape([10164, 1]),
 TensorShape([850, 1])]

In [126]:
for v in vect:
    v = tf.reshape(v, (-1))
    print(tf.slice(v, [0], [10]))

tf.Tensor(
[-0.26260594  0.05045616  0.14847375  0.34235522  0.0350243   0.29450238
 -0.16412902  0.12851956  0.26572266  0.20774849], shape=(10,), dtype=float32)
tf.Tensor(
[-0.01233123  0.04869561  0.03296651  0.0364034   0.08748352 -0.00524808
 -0.08807354  0.20188634  0.03940699  0.09373177], shape=(10,), dtype=float32)
tf.Tensor(
[-0.03111342 -0.18106097  0.13798894  0.1205007  -0.05764659 -0.1340255
 -0.19832337 -0.04008258 -0.05186015 -0.11022102], shape=(10,), dtype=float32)
tf.Tensor(
[ 0.16204588 -0.1678768   0.1572961   0.15544422 -0.25573942  0.07330917
 -0.13756572 -0.06489266  0.21713604 -0.09113318], shape=(10,), dtype=float32)
tf.Tensor(
[-0.3712538   0.1432257   0.14423928 -0.32695284  0.2935225  -0.05299498
 -0.15595293  0.1571002   0.06839338  0.20857866], shape=(10,), dtype=float32)


## Normaliser le Vecteur

In [127]:
v, alpha, beta = scale_function(vect, bucket_size = 128)
len(v)

5

### Effectuer une quantification uniforme

In [132]:
Q = uniform_quantification(v, 8)
Q

[<tf.Tensor: shape=(156,), dtype=float32, numpy=
 array([0.00390625, 0.        , 0.00390625, 0.00390625, 0.00390625,
        0.00390625, 0.        , 0.        , 0.        , 0.        ,
        0.00390625, 0.        , 0.00390625, 0.        , 0.00390625,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.00390625, 0.        , 0.00390625, 0.00390625,
        0.00390625, 0.00390625, 0.        , 0.00390625, 0.        ,
        0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625,
        0.        , 0.        , 0.        , 0.00390625, 0.        ,
        0.        , 0.00390625, 0.        , 0.        , 0.        ,
        0.00390625, 0.00390625, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.00390625, 0.00390625, 0.        ,
        0.00390625, 0.        , 0.        , 0.        , 0.00390625,
        0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.        ,
        0.00390625, 0.00390625, 0.00390625, 0.        , 0.00390625,

### Repasser les poids sous forme matricielle

### Appliquer les poids au model

### Evaluer les performances du model quantifié

In [188]:
from sys import getsizeof
t = np.ones(100, dtype = "uint8")*1000
tab = (t/10).astype("uint8")
tab - 101

array([255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255], dtype=uint8)

In [175]:
getsizeof(t)

212

In [173]:
t = t.astype("float32")
getsizeof(t)

512